# Dataset schema in OpenSAFELY-TPP

This notebook displays the schema of the OpenSAFELY-TPP database. It is part of the technical documentation of the OpenSAFELY platform to help users understand the underlying data and guide analyses. 

The schema information is read from the `OpenSAFELYSchemaInformation` table, which is refreshed at the same time as the core `S1` SystemOne tables. There are some non-automated steps required to update the schema information when a new table is added to the database &mdash; if you can't see a table that you are expecting to see, speak to TPP.

If you would like to apply to use the OpenSAFELY platform please read our [documentation](https://docs.opensafely.org/), the [principles of the platform](https://www.opensafely.org/about/), and information about our [pilot programme for onboarding external users](https://www.opensafely.org/onboarding-new-users/).

If you want to see the Python code used to create this notebook, you can [view it on GitHub](https://github.com/opensafely/database-notebooks/blob/master/notebooks/database-schema.ipynb).

### Data sources
The core SystmOne primary care datasets are held in the `S1` tables in the OpenSAFELY-TPP database. Other externally-linked data sources are listed below, with the table name given in brackets:

* All positive or negative SARS-CoV2 tests, from SGSS (`SGSS_AllTests_Positive` and `SGSS_AllTests_Negative`)
* First-ever positive or negative SARS-CoV2 test, from SGSS (`SGSS_Positive` and `SGSS_Negative`)
* A&E attendances, from SUS Emergency Care data (`EC`)
* In-patient hospital admissions, from SUS Admitted Patient Care Spells data (`APCS`)
* Out-patient hospital appointments, from SUS (`OPA`)
* Covid-related ICU admissions, from ICNARC (`ICNARC`)
* Covid-related in-hospital deaths, from CPNS (`CPNS`)
* COVID-19 Infection Survey, from ONS (`ONS_CIS`)
* All-cause registered deaths, from ONS (`ONS_Deaths`)
* High cost drugs (`HighCostDrugs`)
* Unique Property Reference Number, used for deriving household variables (`UPRN`)
* Master Patient Index (`MPI`)
* Health and Social Care Worker identification, collected at the point of vaccination (`HealthCareWorker`)

Some of these tables are accompanied by additional tables with further data. For instance, `OPA` contains the core out-patient appointment event data, and is supplemented by the `OPA_Cost`, `OPA_Diag`, `OPA_Proc` tables. See the [data schema notebook](https://github.com/opensafely/database-notebooks/blob/master/notebooks/database-schema.ipynb) for more information. 

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import pyodbc
import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown

import sys
sys.path.append('../lib/')
from functions import *

In [2]:
# get the server credentials from environ.txt

dbconn = os.environ.get('FULL_DATABASE_URL', None).strip('"')

In [3]:
## Import schema data and date

with closing_connection(dbconn) as cnxn:
    table_schema = pd.read_sql("""select * from OpenSAFELYSchemaInformation""", cnxn)

today = date.today()

### Notebook run date

In [4]:
display(Markdown(f"""This notebook was run on {today.strftime('%Y-%m-%d')}.  The information below reflects the state of the OpenSAFELY-TPP as at this date."""))

This notebook was run on 2021-10-08.  The information below reflects the state of the OpenSAFELY-TPP as at this date.

## Table names by source
The table below lists all the data tables available in the OpenSAFELY-TPP database and where the data originate from.

In [5]:
table_names = table_schema[['DataSource', 'TableName']].drop_duplicates().sort_values(['DataSource', 'TableName'])
table_names = table_names[table_names['DataSource']!=""]
display(table_names.reset_index(drop=True).style.set_properties(**{'text-align': 'left'}))

,DataSource,TableName
0,ICNARC,ICNARC
1,NHSD_HIGH_COST_DRUGS,HighCostDrugs
2,NHSE_APCS,APCS
3,NHSE_APCS,APCS_Cost
4,NHSE_APCS,APCS_Der
5,NHSE_CPNS,CPNS
6,NHSE_EC,EC
7,NHSE_EC,EC_AlcoholDrugInvolvement
8,NHSE_EC,EC_Comorbidities
9,NHSE_EC,EC_Cost


## Table Schema

The schema for each table contains the following info:

* `ColumnName`, the column name.
* `ColumnType`, the column type, for example integer, numeric or date &mdash; see [SQL Server _data types_ documentation](https://docs.microsoft.com/en-us/sql/t-sql/data-types/data-types-transact-sql) for more details.
* `Precision`, `Scale` and `MaxLength` &mdash; see [SQL Server _precision, scale, and length_ documentation](https://docs.microsoft.com/en-us/sql/t-sql/data-types/precision-scale-and-length-transact-sql) for more details.
* `IsNullable`, are Null values accepted.

The schema for each table is printed below.

In [6]:
pd.set_option('display.max_columns', None)

for source in table_names['DataSource'].unique():
    
    display(Markdown("\n"))
    display(Markdown(f"### {source}"))
    
    for table in table_names.loc[table_names['DataSource']==source, 'TableName']:
        tab = table_schema[table_schema['TableName']==table]
        tab = tab.drop(columns=['TableName', 'DataSource', 'ColumnId', 'CollationName'])
        display(Markdown(f"#### {table}"))
        display(tab.set_index('ColumnName'))

### ICNARC

#### ICNARC

,ColumnType,MaxLength,Precision,Scale,IsNullable
ColumnName,,,,,
Patient_ID,bigint,8,19,0,False
ICNARC_ID,bigint,8,19,0,True
CalculatedAge,int,4,10,0,True
EstimatedAge,int,4,10,0,True
Sex,varchar,10,0,0,True
OriginalHospitalAdmissionDate,datetime,8,23,3,True
HospitalAdmissionDate,datetime,8,23,3,True
IcuAdmissionDateTime,datetime,8,23,3,True
TransferredIn,varchar,10,0,0,True


### NHSD_HIGH_COST_DRUGS

#### HighCostDrugs

...

,ColumnType,MaxLength,Precision,Scale,IsNullable
ColumnName,,,,,
Patient_ID,bigint,8,19,0,False
Vaccination_ID,bigint,8,19,0,False
VaccinationDate,datetime,8,23,3,False
VaccinationName,varchar,100,0,0,False
VaccinationName_ID,bigint,8,19,0,False
VaccinationSchedulePart,int,4,10,0,False


#### VaccinationReference

,ColumnType,MaxLength,Precision,Scale,IsNullable
ColumnName,,,,,
VaccinationName_ID,int,4,10,0,False
VaccinationName,varchar,100,0,0,False
VaccinationContent,varchar,50,0,0,False


#### YCodeToSnomedMapping

,ColumnType,MaxLength,Precision,Scale,IsNullable
ColumnName,,,,,
YCode,varchar,5,0,0,False
SctConceptId,bigint,8,19,0,True
